In [1]:
f = open("input.txt", "r")
input_hex = f.read()[:-1]
# input_hex = "9C0141080250320F1802104A08"
input_hex[:100]

'E0525D9802FA00B80021B13E2D4260004321DC648D729DD67B2412009966D76C0159ED274F6921402E9FD4AC1B0F652CD339'

In [2]:
data = bin(int(input_hex, 16))[2:]
data = data.zfill(len(input_hex)*4)
data[:100]

'1110000001010010010111011001100000000010111110100000000010111000000000000010000110110001001111100010'

In [3]:
import math

LEN_OP0_PACK = 22
LEN_OP1_PACK = 18

def bin_to_int(x):
    return int(x, 2)

def get_version(x):
    version = bin_to_int(x[:3])
    # print(f"{version=}")
    return version

def get_operator(x):
    operator = bin_to_int(x[3:6])
    # print(f"{operator=}")
    return operator

def get_operator_id(x):
    operator_l = int(x[6])
    # print(f"{operator_l=}")
    return operator_l

def get_operator_value(x):
    operator_v = bin_to_int(x[7:LEN_OP0_PACK])
    # print(f"{operator_v=}, {x[7:LEN_OP0_PACK]}, {x}")
    return operator_v

def convert_operator_value(x, values):
    if not values:
        print(x)
        return
    if x==0: 
        # print('sum', values)
        return sum(values)
    if x==1: 
        # print('*', values)
        return math.prod(values)
    if x==2:         
        # print('min', values)
        return min(values)
    if x==3: 
        # print('max', values)
        return max(values)
    if x==5: 
        # print('>', values)
        return int(values[0] > values[1])
    if x==6: 
        # print('<', values)
        return int(values[0] < values[1])
    if x==7: 
        # print('==', values)
        return int(values[0] == values[1])

def get_operator_subpackets(x):
    operator_subs = bin_to_int(x[7:LEN_OP1_PACK])
    return operator_subs

def get_literal(x):
    lit_len = 6
    lit_value = x[lit_len+1:lit_len+5]
    while int(x[lit_len:][0]) != 0:
        lit_len+=5
        lit_value += x[lit_len+1:lit_len+5]
    lit_value = bin_to_int(lit_value)
    lit_len += 5
    return lit_len, lit_value


versions = []
def retract(data, i):
    if len(set(data[i:])) < 2:
        print('EOF') 
    versions.append(get_version(data[i:]))
    if (op := get_operator(data[i:])) == 4:
        lit_len, lit_value = get_literal(data[i:])
        return i+lit_len, lit_value
    else:   
        values = []
        if get_operator_id(data[i:]) == 0:
            operator_value = get_operator_value(data[i:])
            i += LEN_OP0_PACK
            sub_len = i + operator_value
            while i<sub_len:
                i, val = retract(data, i)
                values.append(val) 
        else:
            subs = get_operator_subpackets(data[i:])
            i += LEN_OP1_PACK
            for _ in range(subs): 
                i, val = retract(data, i)
                values.append(val)
        return i, convert_operator_value(op, values)

i, val = (retract(data, 0))
print()
print('TOTAL:', val)
print('VERSIONS:', sum(versions))


TOTAL: 13476220616073
VERSIONS: 940
